In [1]:
import pandas as pd
import numpy as np
from pycaret.utils import check_metric
from pycaret.classification import *

The Predicted_Churn and Score columns are added onto the dataset where:


1. **Predicted_Churn** is the prediction (where 1 = churn, 0 = not churn)
2. **Score** is the probability of the prediction. 

In [2]:
data_predictions = pd.read_csv('data_predictions.csv')
data_predictions.rename(columns = {'Label': 'Predicted_Churn'}, inplace=True)
data_predictions.head()

,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,churn,Predicted_Churn,Score
0,lmkebamcaaclubfxadlmueccxoimlema,287252,15063,25744,5212.75,4723,0.0,143.80,0.164057,0.100243,45.806878,t,499.55,37.56,37.56,2,515.93,6,lxidpiddsbxsbosboudacockeimpuepw,33.000,0.167699,0.105428,0.074570,44.444710,24.437330,16.291555,0,0,0.9813
1,foosdfpfkusacimwkcsosbicdxkicaua,8296,0,48,701.36,0,0.0,16.67,0.116900,0.000000,40.939027,f,0.00,15.45,15.24,1,72.30,4,ldkssxwpmemidmecebumciepifcamkci,13.200,0.127710,0.000000,0.000000,40.728885,0.000000,0.000000,0,0,0.9999
2,foosdfpfkusacimwkcsosbicdxkicaua,37402,0,3026,2254.76,1508,0.0,130.35,0.145711,0.098142,44.311378,f,170.26,27.14,27.14,1,227.31,6,lxidpiddsbxsbosboudacockeimpuepw,17.321,0.150211,0.100505,0.071536,44.266931,24.339581,16.226389,0,0,0.9999
3,foosdfpfkusacimwkcsosbicdxkicaua,9183,0,201,766.01,0,0.0,16.98,0.116910,0.000000,41.105201,f,0.00,16.44,16.44,1,78.33,4,ldkssxwpmemidmecebumciepifcamkci,13.200,0.129300,0.000000,0.000000,40.728885,0.000000,0.000000,0,0,0.9999
4,foosdfpfkusacimwkcsosbicdxkicaua,9551,0,258,793.55,0,0.0,17.21,0.116910,0.000000,41.271364,f,0.00,16.77,16.66,1,81.08,4,ldkssxwpmemidmecebumciepifcamkci,13.200,0.129444,0.000000,0.000000,41.063970,0.000000,0.000000,0,0,0.9999


## 1. Evaluating the performance of the model on the data
Below is an overview of the performance of the model on the full data. The model is perfect, it only misclassified 16 retained customers as churned, other than that it performs very well 99% of the time.

In [3]:
pd.crosstab(data_predictions['churn'], data_predictions['Predicted_Churn'])

Predicted_Churn,0,1
churn,,
0,146903,16
1,0,18098


#### 1.1 Accuracy
it is simply a ratio of correctly predicted observation to the total observations. The model has a 99% accuracy.

In [4]:
check_metric(data_predictions['churn'], data_predictions['Predicted_Churn'], metric = 'Accuracy')

0.9999

#### 1.2 Precision
The ratio of correctly predicted positive observations to the total predicted positive observations. The model has a 99% precision.

In [5]:
check_metric(data_predictions['churn'], data_predictions['Predicted_Churn'], metric = 'Precision')

0.9991

#### 1.3 Recall (a.k.a Sensitivity)
The ratio of correctly predicted positive observations to the all observations in actual class

In [6]:
check_metric(data_predictions['churn'], data_predictions['Predicted_Churn'], metric = 'Recall')

1.0

## 2. Business Impact

**The SME division head proposed that we give a 20% discount to high propensity-to-churn customers.**

However we need a cut-off to implement this, for this study, i used 75%. In other words, we will offer the discount to customers with 75% or higher probability of churning.

**Assumption:** Everyone who is offered a discount will accept it.

In [7]:
data_predictions['revenue'] = data_predictions['forecast_cons_12m'] * data_predictions['forecast_price_energy_p1'] 
data_new = data_predictions[['churn','Predicted_Churn', 'Score', 'revenue']]

In [8]:
def churn_cutoff(df, cutoff=0.75, churn='Predicted_Churn', score='Score'):
    df = df[df[churn] == 1]
    df = df[df[score] >= cutoff]
    df = df.sort_values(by='Score', ascending=False)
    df = df.reset_index(drop=True)
    return df

In [9]:
data_churn = churn_cutoff(data_new)
data_churn.shape

(18114, 4)

In [10]:
data_churn.head()

,churn,Predicted_Churn,Score,revenue
0,1,1,0.9998,5502.921301
1,1,1,0.9998,1326.584700
2,1,1,0.9998,2619.068816
3,1,1,0.9998,1087.056729
4,1,1,0.9998,1597.239075


It is important to note that almost all the customers who churned are certainly predicted to churn by the model (Score>=0.95).

In [11]:
print("Percentage of revenue spent on discount strategy for churning customers: ",
      round(data_churn['revenue'].sum()*0.2/data_predictions['revenue'].sum(),2)*100)

Percentage of revenue spent on discount strategy for churning customers:  11.0


The company will spend 11% of its revenue on offering a 20% discount to **high-propensity-to-churn** customers.
This is not the best strategy as the company as the cost is too high.